In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from medmnist import PneumoniaMNIST
import medmnist
from sklearn.metrics import precision_score, recall_score, f1_score

# Ensure the dataset is the latest version
medmnist.INFO

# Load PneumoniaMNIST dataset
def load_pneumonia_mnist():
    train_dataset = PneumoniaMNIST(split="train", download=True, transform=None)
    val_dataset = PneumoniaMNIST(split="val", download=True, transform=None)
    test_dataset = PneumoniaMNIST(split="test", download=True, transform=None)
    
    X_train = np.concatenate((train_dataset.imgs, val_dataset.imgs), axis=0)
    Y_train = np.concatenate((train_dataset.labels.flatten(), val_dataset.labels.flatten()), axis=0)
    X_test = test_dataset.imgs
    Y_test = test_dataset.labels.flatten()
    
    return X_train, X_test, Y_train, Y_test

# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 3 * 3, 128)
        self.fc2 = nn.Linear(128, 2)
    
    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = self.pool3(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 3 * 3)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Function to calculate the number of parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Load data
X_train, X_test, Y_train, Y_test = load_pneumonia_mnist()
X_train = X_train.astype(np.float32) / 255.0
X_test = X_test.astype(np.float32) / 255.0

# Create DataLoaders
batch_size = 64
train_loader = DataLoader(TensorDataset(torch.tensor(X_train).unsqueeze(1), torch.tensor(Y_train)), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(torch.tensor(X_test).unsqueeze(1), torch.tensor(Y_test)), batch_size=batch_size, shuffle=False)

# Initialize the model, loss function, and optimizer
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Print the number of parameters
num_params = count_parameters(model)
print(f'The model has {num_params} trainable parameters')

# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_accuracy = 100 * correct / total
    print(f'Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}, Training Accuracy: {train_accuracy}%')

# Evaluation
model.eval()
y_true = []
y_pred = []
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        y_true.extend(labels.tolist())
        y_pred.extend(predicted.tolist())

test_accuracy = 100 * correct / total
precision = precision_score(y_true, y_pred, average='binary')
recall = recall_score(y_true, y_pred, average='binary')
f1 = f1_score(y_true, y_pred, average='binary')

print(f'Test Accuracy: {test_accuracy}%')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


Using downloaded and verified file: C:\Users\quaso\.medmnist\pneumoniamnist.npz
Using downloaded and verified file: C:\Users\quaso\.medmnist\pneumoniamnist.npz
Using downloaded and verified file: C:\Users\quaso\.medmnist\pneumoniamnist.npz
The model has 240514 trainable parameters
Epoch 1, Loss: 0.4118941900570218, Training Accuracy: 80.63837920489297%
Epoch 2, Loss: 0.15106550395125296, Training Accuracy: 93.86467889908256%
Epoch 3, Loss: 0.1275502635029758, Training Accuracy: 95.2408256880734%
Epoch 4, Loss: 0.10825500489644162, Training Accuracy: 95.89067278287462%
Epoch 5, Loss: 0.09754234890839676, Training Accuracy: 96.3493883792049%
Epoch 6, Loss: 0.08663631360069281, Training Accuracy: 96.76987767584097%
Epoch 7, Loss: 0.07904060396206815, Training Accuracy: 97.01834862385321%
Epoch 8, Loss: 0.06858732195843648, Training Accuracy: 97.55351681957187%
Epoch 9, Loss: 0.06382556954716764, Training Accuracy: 97.84021406727828%
Epoch 10, Loss: 0.06783381740494472, Training Accuracy: 